In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns
#import geopandas as gpd

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/ranking-of-united-states-cities-by-area/Ranking of United States cities by area.csv')

**Анализируем данные**

In [ ]:
df.head()

In [ ]:
df.info()

Исходя из информации видно, что в датасете **нет пропущенных данных**

Однако числовые данные (площадь и население) представлены в виде категориальных данных. Для анализа в последаствии изменим тип представленных данных

In [ ]:
# Просмотрим названия штатов, чтобы не встречались сокращения и т.п.
df['State '].unique()

In [ ]:
# Так как анализ площадей буду производить на основании кв. километров, то значения в милях удаляем
df.drop(labels=['Land area (sq mi)', 'Water area (sq mi)', 'Total area (sq mi)'], axis=1, inplace=True)

In [ ]:
# Для удобства переименовываем названия категорий (столбцов)
df = df.rename(columns={'City ': 'City', 'State ': 'State', ' (km2)': 'Land area, km2', ' (km2).1': 'Water area, km2', ' (km2).2': 'Total area, km2'})

In [ ]:
# Убираем ',' чтобы не мешали
df['Land area, km2'] = df['Land area, km2'].apply(lambda x: x.replace(',',''))
df['Water area, km2'] = df['Water area, km2'].apply(lambda x: x.replace(',',''))
df['Total area, km2'] = df['Total area, km2'].apply(lambda x: x.replace(',',''))
df['Population (2020)'] = df['Population (2020)'].apply(lambda x: x.replace(',',''))

In [ ]:
# Изменяем тип данных числовых критериев
df['Land area, km2'] = df['Land area, km2'].astype('float64')
df['Water area, km2'] = df['Water area, km2'].astype('float64')
df['Total area, km2'] = df['Total area, km2'].astype('float64')
df['Population (2020)'] = df['Population (2020)'].astype('int64')

# Анализ данных о городах в общем (в масштабах страны)

**Топ 10 городов страны по количеству жителей**

In [ ]:
df = df.sort_values(by='Population (2020)', ascending=False)
df_10_pop = df.head(10)

In [ ]:
plt.figure(figsize=(13, 6))
plots = sns.barplot(x='City', y='Population (2020)', data=df_10_pop) 

for bar in plots.patches: 
    plots.annotate(format(bar.get_height().astype(int)),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=10, xytext=(0, 8),
                   textcoords='offset points')

plt.ylabel('Population')
plt.title('Population of Top 10 Cities')
plt.show()

**Топ 10 городов страны по площади земли**

In [ ]:
df = df.sort_values(by='Land area, km2', ascending=False)
df_10_land = df.head(10)

In [ ]:
plt.figure(figsize=(13, 6))
plots = sns.barplot(x='City', y='Land area, km2', data=df_10_land) 

for bar in plots.patches: 
    plots.annotate(format(bar.get_height().astype(int)),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=10, xytext=(0, 8),
                   textcoords='offset points')

plt.ylabel('Land area')
plt.title('Land area of Top 10 Cities')
plt.show()

**Топ 10 городов страны по площади воды**

In [ ]:
df = df.sort_values(by='Water area, km2', ascending=False)
df_10_water = df.head(10)

In [ ]:
plt.figure(figsize=(13, 6))
plots = sns.barplot(x='City', y='Water area, km2', data=df_10_water) 

for bar in plots.patches: 
    plots.annotate(format(bar.get_height().astype(int)),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=10, xytext=(0, 8),
                   textcoords='offset points')

plt.ylabel('Water area')
plt.title('Water area of Top 10 Cities')
plt.show()

**Топ 10 городов страны по общей площади**

In [ ]:
df = df.sort_values(by='Total area, km2', ascending=False)
df_10_total = df.head(10)

In [ ]:
plt.figure(figsize=(13, 6))
plots = sns.barplot(x='City', y='Total area, km2', data=df_10_total) 

for bar in plots.patches: 
    plots.annotate(format(bar.get_height().astype(int)),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=10, xytext=(0, 8),
                   textcoords='offset points')

plt.ylabel('Total area')
plt.title('Total area of Top 10 Cities')
plt.show()

# Анализируем показатели (критерии) относительно штатов

Для этого суммироуем данные городов для каждого штата

In [ ]:
# Берем все названия (значения) штатов и создаем датафрейм с одним этим столбцом (критерием)
d_state = {'State' : df['State'].unique()}
df_state = pd.DataFrame(data=d_state, index=df['State'].unique())

In [ ]:
# В созданный датафрейм добавляем суммарные данные по каждому штату
for col in df.columns[3:]:
    df_state[col] = [df.loc[df['State'] == val, col].sum() for val in df_state['State']]

In [ ]:
df_state

**Анализ по числу жителей**

In [ ]:
df_state = df_state.sort_values(by='Population (2020)', ascending=False)

plot = df_state.plot.pie(y='Population (2020)', figsize=(13, 13))

In [ ]:
# Для удобства представления берем только топ 10 значений
plt.figure(figsize=(13, 6))
plots = sns.barplot(x='State', y='Population (2020)', data=df_state.head(10)) 

for bar in plots.patches: 
    plots.annotate(format(bar.get_height().astype(int)),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=10, xytext=(0, 8),
                   textcoords='offset points')

plt.ylabel('Population')
plt.title('Population of Top 10 States')
plt.show()

**Анализ по площади земли**

In [ ]:
df_state = df_state.sort_values(by='Land area, km2', ascending=False)

plot = df_state.plot.pie(y='Land area, km2', figsize=(13, 13))

In [ ]:
plt.figure(figsize=(13, 6))
plots = sns.barplot(x='State', y='Land area, km2', data=df_state.head(10)) 

for bar in plots.patches: 
    plots.annotate(format(bar.get_height().astype(int)),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=10, xytext=(0, 8),
                   textcoords='offset points')

plt.ylabel('Land area')
plt.title('Land area of Top 10 States')
plt.show()

**Анализ по площади воды**

In [ ]:
df_state = df_state.sort_values(by='Water area, km2', ascending=False)

plot = df_state.plot.pie(y='Water area, km2', figsize=(13, 13))

In [ ]:
plt.figure(figsize=(13, 6))
plots = sns.barplot(x='State', y='Water area, km2', data=df_state.head(10)) 

for bar in plots.patches: 
    plots.annotate(format(bar.get_height().astype(int)),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=10, xytext=(0, 8),
                   textcoords='offset points')

plt.ylabel('Water area')
plt.title('Water area of Top 10 States')
plt.show()

**Анализ по общей площади**

In [ ]:
df_state = df_state.sort_values(by='Total area, km2', ascending=False)

plot = df_state.plot.pie(y='Total area, km2', figsize=(13, 13))

In [ ]:
plt.figure(figsize=(13, 6))
plots = sns.barplot(x='State', y='Total area, km2', data=df_state.head(10)) 

for bar in plots.patches: 
    plots.annotate(format(bar.get_height().astype(int)),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=10, xytext=(0, 8),
                   textcoords='offset points')

plt.ylabel('Total area')
plt.title('Total area of Top 10 States')
plt.show()

# Анализ Топ города для каждого штата

В каждом штате выделим один город по определенному признаку:

1. Наибольшее население
2. Наибольшая площать земли
3. Наибольшая площать воды
4. Наибольшая общая площать

In [ ]:
d = {'City':[], 'State':[], 'Land area, km2':[], 'Water area, km2':[], 'Total area, km2':[], 'Population (2020)':[]}
df_st = pd.DataFrame(data=d, columns=['City', 'State', 'Land area, km2', 'Water area, km2', 'Total area, km2', 'Population (2020)'])
df_st_pop = pd.DataFrame(data=d, columns=['City', 'State', 'Land area, km2', 'Water area, km2', 'Total area, km2', 'Population (2020)'])
df_st_l_ar = pd.DataFrame(data=d, columns=['City', 'State', 'Land area, km2', 'Water area, km2', 'Total area, km2', 'Population (2020)'])
df_st_w_ar = pd.DataFrame(data=d, columns=['City', 'State', 'Land area, km2', 'Water area, km2', 'Total area, km2', 'Population (2020)'])
df_st_t_ar = pd.DataFrame(data=d, columns=['City', 'State', 'Land area, km2', 'Water area, km2', 'Total area, km2', 'Population (2020)'])


In [ ]:
# Для каждой категории создаем отдельный датафрейм
for state in df['State'].unique():
    for str_data in df:
        df_st = df.loc[df['State'] == state]
    df_st_pop = df_st_pop.append(df_st.loc[df_st['Population (2020)'] == df_st['Population (2020)'].max()])
    df_st_l_ar = df_st_l_ar.append(df_st.loc[df_st['Land area, km2'] == df_st['Land area, km2'].max()])
    df_st_w_ar = df_st_w_ar.append(df_st.loc[df_st['Water area, km2'] == df_st['Water area, km2'].max()])
    df_st_t_ar = df_st_t_ar.append(df_st.loc[df_st['Total area, km2'] == df_st['Total area, km2'].max()])

In [ ]:
df_st_pop = df_st_pop.sort_values(by='Population (2020)', ascending=False)
df_st_pop.head(10)

In [ ]:
df_st_l_ar = df_st_l_ar.sort_values(by='Land area, km2', ascending=False)
df_st_l_ar.head(10)

In [ ]:
df_st_w_ar = df_st_w_ar.sort_values(by='Water area, km2', ascending=False)
df_st_w_ar.head(10)

In [ ]:
df_st_t_ar = df_st_t_ar.sort_values(by='Total area, km2', ascending=False)
df_st_t_ar.head(10)

# Вывод

В ходе выполнения анализа получено:

1. Для городов:
* Город с самым большим населением - New York City (штат New York)
* Город с самой большой площадью земли - Sitka (штат Alaska)
* Город с самой большой площадью воды - Sitka (штат Alaska)
* Город с самой большой общей площадью - Sitka (штат Alaska)

2. Для штатов:
* Штат с самым большим населением - Texas
* Штат с самой большой площадью земли - Alaska
* Штат с самой большой площадью воды - Alaska
* Штат с самой большой общей площадью - Alaska